<a href="https://colab.research.google.com/github/armandoordonez/GenAI/blob/main/Lab_2_fine_tune_generative_ai_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tune un modelo de Gen AI Model para resúmenes

En este cuaderno, se hará en fine-tune de un LLM existente de Hugging Face para mejorar el resumen de los diálogos. Utilizará el modelo [FLAN-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5), que proporciona un modelo tuneado con instrucciones de alta calidad y puede resumir texto. Para mejorar las inferencias, se usará full fine-tuning y evaluará los resultados con métricas de ROUGE. Luego, se usará Parameter Efficient Fine-Tuning (PEFT), evaluará el modelo resultante y verá que los beneficios de PEFT superan las desventajas de rendimiento.

# Tabla de contenido

- [ 1 - Set up Kernel, Load Required Dependencies, Dataset and LLM](#1)
  - [ 1.1 - Set up Kernel and Required Dependencies](#1.1)
  - [ 1.2 - Load Dataset and LLM](#1.2)
  - [ 1.3 - Test the Model with Zero Shot Inferencing](#1.3)
- [ 2 - Perform Full Fine-Tuning](#2)
  - [ 2.1 - Preprocess the Dialog-Summary Dataset](#2.1)
  - [ 2.2 - Fine-Tune the Model with the Preprocessed Dataset](#2.2)
  - [ 2.3 - Evaluate the Model Qualitatively (Human Evaluation)](#2.3)
  - [ 2.4 - Evaluate the Model Quantitatively (with ROUGE Metric)](#2.4)
- [ 3 - Perform Parameter Efficient Fine-Tuning (PEFT)](#3)
  - [ 3.1 - Setup the PEFT/LoRA model for Fine-Tuning](#3.1)
  - [ 3.2 - Train PEFT Adapter](#3.2)
  - [ 3.3 - Evaluate the Model Qualitatively (Human Evaluation)](#3.3)
  - [ 3.4 - Evaluate the Model Quantitatively (with ROUGE Metric)](#3.4)

<a name='1'></a>
## 1 - Set up Kernel, Load Required Dependencies, Dataset and LLM

In [1]:
!python --version

Python 3.10.12


In [2]:
# Opción 1: Instalación paso a paso con comandos mágicos
print("🚀 Instalando PyTorch...")
!pip install torch==2.8.0+cpu torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

print("🚀 Instalando Transformers...")
!pip install transformers==4.56.1

print("🚀 Instalando Datasets...")
!pip install datasets==4.0.0

print("🚀 Instalando Evaluate...")
!pip install evaluate==0.4.5

print("🚀 Instalando ROUGE Score...")
!pip install rouge_score==0.1.2

print("🚀 Instalando PEFT...")
!pip install peft==0.17.1

print("🚀 Instalando dependencias adicionales...")
!pip install accelerate huggingface_hub safetensors

print("✨ ¡Instalación completada!")

# Verificar instalaciones
print("\n🔍 Verificando instalaciones...")
try:
    import torch
    import transformers
    import datasets
    import evaluate
    import peft
    
    print("✅ Verificación exitosa:")
    print(f"   PyTorch: {torch.__version__}")
    print(f"   Transformers: {transformers.__version__}")
    print(f"   Datasets: {datasets.__version__}")
    print(f"   PEFT: {peft.__version__}")
    print("\n🎉 ¡Todo listo para hacer fine-tuning!")
    
except ImportError as e:
    print(f"❌ Error en la verificación: {e}")
    print("Puede que necesites reiniciar el kernel del notebook.")

🚀 Instalando PyTorch...
Looking in indexes: https://download.pytorch.org/whl/cpu

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
🚀 Instalando Transformers...

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
🚀 Instalando Datasets...

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
🚀 Instalando Evaluate...

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
🚀 Instalando ROUGE Score...

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
🚀 Instalando PEFT...

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
🚀 Instalando dependencias adicionales...

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run:

/home/pablo/Desktop/IA2/mi_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Verificación exitosa:
   PyTorch: 2.8.0+cpu
   Transformers: 4.56.1
   Datasets: 4.0.0
   PEFT: 0.17.1

🎉 ¡Todo listo para hacer fine-tuning!


In [3]:
!pip show torch torchdata transformers datasets evaluate rouge_score loralib peft

Name: torch
Version: 2.8.0+cpu
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /home/pablo/Desktop/IA2/mi_env/lib/python3.10/site-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: accelerate, peft, torchaudio, torchvision
---
Name: transformers
Version: 4.56.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /home/pablo/Desktop/IA2/mi_env/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required

<a name='1.1'></a>
### 1.1 - Set up Kernel and Required Dependencies

In [4]:
# Importe los componentes necesarios.
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

<a name='1.2'></a>
### 1.2 - Cargar Dataset y LLM

[Finance-Instruct-500k](https://huggingface.co/datasets/Josephgflowers/Finance-Instruct-500k) es un dataset de Hugging Face especializado en instrucciones financieras. Contiene más de 500,000 ejemplos de instrucciones financieras con sus respectivas respuestas, perfecto para fine-tuning con LoRA.

In [5]:
dataset = load_dataset("Josephgflowers/Finance-Instruct-500k", split="train[:10000]")  # Solo primeros 10K ejemplos

# Convertir a DatasetDict si necesitas splits
from datasets import DatasetDict
dataset = DatasetDict({"train": dataset})

In [6]:
print(dataset["train"].features)

{'system': Value('string'), 'user': Value('string'), 'assistant': Value('string')}


Cargamos el modelo pre-entrenado [T5-small](https://huggingface.co/t5-small) y su tokenizador directamente desde HuggingFace. T5-small es una versión más liviana con solo 60M parámetros, ideal para fine-tuning rápido con LoRA.

In [7]:
# Cargamos el modelo y el tokenizador

model_name='google/flan-t5-small'
# model_name='t5-google/flan-t5-small'
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Es posible extraer la cantidad de parámetros del modelo y descubrir cuántos de ellos se pueden entrenar. 

In [8]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"Parametros entrenables del modelo: {trainable_model_params:,.0f}\n Total de parametros del modelo: {all_model_params:,.0f}\n Porcentaje de parametros entrenables {100 * trainable_model_params / all_model_params:.0f}%"
#print(f'El area es: {area:,.2f}')
print(print_number_of_trainable_model_parameters(original_model))

Parametros entrenables del modelo: 76,961,152
 Total de parametros del modelo: 76,961,152
 Porcentaje de parametros entrenables 100%


In [9]:
# Verificar la estructura del dataset Finance-Instruct-500k
print("🔍 Explorando el dataset Finance-Instruct-500k...")
print(f"Splits disponibles: {list(dataset.keys())}")
print(f"Tipo de dataset: {type(dataset)}")

# Ver información de cada split disponible
for split_name in dataset.keys():
    print(f"\n📊 Split '{split_name}':")
    print(f"   - Cantidad de ejemplos: {len(dataset[split_name])}")
    print(f"   - Columnas: {list(dataset[split_name].features.keys())}")

# Mostrar un ejemplo del primer split disponible
first_split = list(dataset.keys())[0]
print(f"\n📝 Ejemplo del split '{first_split}':")
print(f"   - user: {dataset[first_split][0]['user'][:100]}...")
print(f"   - assistant: {dataset[first_split][0]['assistant'][:100]}...")

🔍 Explorando el dataset Finance-Instruct-500k...
Splits disponibles: ['train']
Tipo de dataset: <class 'datasets.dataset_dict.DatasetDict'>

📊 Split 'train':
   - Cantidad de ejemplos: 10000
   - Columnas: ['system', 'user', 'assistant']

📝 Ejemplo del split 'train':
   - user: Explain tradeoffs between fiscal and monetary policy as tools in a nation's economic toolkit. Provid...
   - assistant: Fiscal and monetary policy are the two main tools that governments have to influence economic activi...


In [10]:
# Dividir manualmente el dataset si solo tiene 'train'
print("🛠️ Creando divisiones manuales del dataset...")

if len(dataset.keys()) == 1 and 'train' in dataset.keys():
    print("El dataset solo tiene 'train'. Dividiendo manualmente...")
    
    # Obtener el total de ejemplos
    total_examples = len(dataset['train'])
    print(f"Total de ejemplos: {total_examples:,}")
    
    # Definir índices para cada split (puedes ajustar estos porcentajes)
    train_end = int(0.7 * total_examples)      # 70% para entrenamiento
    val_end = int(0.85 * total_examples)       # 15% para validación  
    # El resto (15%) será para test
    
    print(f"📊 División propuesta:")
    print(f"   - Train: 0 a {train_end:,} ({70}%)")
    print(f"   - Validation: {train_end:,} a {val_end:,} ({15}%)")  
    print(f"   - Test: {val_end:,} a {total_examples:,} ({15}%)")
    
    # Crear los nuevos splits manualmente
    train_data = dataset['train'].select(range(0, train_end))
    val_data = dataset['train'].select(range(train_end, val_end))
    test_data = dataset['train'].select(range(val_end, total_examples))
    
    # Crear el nuevo diccionario de dataset
    from datasets import DatasetDict
    dataset = DatasetDict({
        'train': train_data,
        'validation': val_data,
        'test': test_data
    })
    
    print("✅ Dataset dividido exitosamente!")
    print(f"   - Train: {len(dataset['train']):,} ejemplos")
    print(f"   - Validation: {len(dataset['validation']):,} ejemplos")
    print(f"   - Test: {len(dataset['test']):,} ejemplos")

else:
    print("El dataset ya tiene múltiples splits:")
    for split_name in dataset.keys():
        print(f"   - {split_name}: {len(dataset[split_name]):,} ejemplos")

🛠️ Creando divisiones manuales del dataset...
El dataset solo tiene 'train'. Dividiendo manualmente...
Total de ejemplos: 10,000
📊 División propuesta:
   - Train: 0 a 7,000 (70%)
   - Validation: 7,000 a 8,500 (15%)
   - Test: 8,500 a 10,000 (15%)
✅ Dataset dividido exitosamente!
   - Train: 7,000 ejemplos
   - Validation: 1,500 ejemplos
   - Test: 1,500 ejemplos


<a name='1.3'></a>
### 1.3 - Prueba del modelo con Zero Shot Inferencing

Pruebe el modelo con la inferencia de tiro cero. Puede ver que el modelo tiene dificultades para resumir el diálogo en comparación con el resumen de referencia, pero extrae información importante del texto que indica que el modelo se puede ajustar a la tarea en cuestión.

In [11]:
index = 200

# Extraemos la instrucción y respuesta del nuevo dataset
instruction = dataset['test'][index]['user']
expected_response = dataset['test'][index]['assistant']

prompt = f"""
financial instruction: {instruction}

Answer:
"""
inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True 
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f' PROMPT DE ENTRADA:\n{prompt}')
print(dash_line)
print(f'RESPUESTA HUMANA (BASELINE) :\n{expected_response}\n')
print(dash_line)
print(f'RESPUESTA GENERADA POR EL MODELO CON ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
 PROMPT DE ENTRADA:

financial instruction: As a math student with a background in applied mathematics and numerics, how can I transition into the field of mathematical finance without prior knowledge in finance or economics?

Answer:

---------------------------------------------------------------------------------------------------
RESPUESTA HUMANA (BASELINE) :
To transition into mathematical finance without prior knowledge in the field, consider the following steps:

* Acquire foundational knowledge: Begin by reading introductory books on finance and economics, such as "Investment Science" by Luenberger.
* Study mathematical finance: Enroll in courses or read books on mathematical finance, such as "Stochastic Calculus for Finance" by Steven Shreve or "Monte Carlo Methods in Financial Engineering" by Glasserman.
* Build a strong statistical foundation: Focus on mastering multivariate s

<a name='2'></a>
## 2 - Realizar Full Fine-Tuning

<a name='2.1'></a>
### 2.1 - Preprocesar el dataset Dialog-Summary

Se necesita convertir los pares dialogo-resumen (prompt-response) en instrucciones explícitas para el LLM. Agregar una instrucción al inicio del dialogo como `Summarize the following conversation` y al inicio del resumen agregar `Summary`como se muestra a continuación:

Training prompt (dialogue):
```
Summarize the following conversation.

    Chris: This is his part of the conversation.
    Antje: This is her part of the conversation.
    
Summary:
```

Training response (summary):
```
Both Chris and Antje participated in the conversation.
```

Then preprocess the prompt-response dataset into tokens and pull out their `input_ids` (1 per token).

In [12]:
def tokenize_function(example):
    # El dataset Finance-Instruct-500k tiene 'user' (instrucción) y 'assistant' (respuesta)
    # Formato: "Instrucción: <user>\n\nRespuesta: "
    start_prompt = 'Instrucción financiera: '
    end_prompt = '\n\nRespuesta: '
    
    # Construimos el prompt para cada ejemplo en el batch
    prompt = [start_prompt + instruction + end_prompt for instruction in example["user"]]
    print("Size of prompt list: ", len(prompt))  # Imprimir el tamaño de la lista
    
    # Tokenizamos los prompts (inputs)
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    
    # Tokenizamos las respuestas (labels)
    example['labels'] = tokenizer(example["assistant"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# El dataset Finance-Instruct-500k contiene 3 splits: train, validation, test
# La función tokenize_function procesa todos los datos en batches

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Removemos las columnas originales, manteniendo solo input_ids y labels
tokenized_datasets = tokenized_datasets.remove_columns(['system', 'user', 'assistant'])

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Size of prompt list:  1000


Map:  67%|██████▋   | 1000/1500 [00:01<00:00, 677.48 examples/s]

Size of prompt list:  500


Map: 100%|██████████| 1500/1500 [00:02<00:00, 664.90 examples/s]


In [13]:
print (type(tokenized_datasets))
first_example = tokenized_datasets["train"].select([0])
print(first_example)
print(first_example['labels'])

<class 'datasets.dataset_dict.DatasetDict'>
Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 1
})
Column([[29169, 11, 3, 14356, 1291, 33, 8, 192, 711, 1339, 24, 10524, 43, 12, 2860, 1456, 1756, 5, 328, 284, 43, 1393, 11, 3314, 1549, 7, 5, 29169, 1291, 2401, 7, 12, 789, 2887, 11, 1104, 257, 3055, 5, 19119, 13, 5043, 1291, 560, 10, 1697, 3, 2092, 8, 1651, 419, 19988, 6, 8, 412, 5, 134, 5, 789, 6960, 3, 9, 5043, 26544, 190, 8, 797, 16532, 11, 419, 15601, 297, 1983, 13, 10425, 100, 1285, 1936, 2887, 30, 3620, 6, 1104, 8620, 6, 11, 8148, 17646, 1393, 5, 37, 8762, 47, 12, 4888, 12955, 2173, 11, 14954, 1456, 1756, 5, 6536, 43, 435, 24, 8, 26544, 141, 3, 9, 1465, 68, 11306, 1113, 30, 1456, 1170, 5, 1697, 37, 2523, 3602, 6960, 1104, 8620, 16, 1233, 28, 8, 1288, 13, 3094, 268, 1729, 11, 1170, 5, 611, 6, 8, 1504, 30, 1170, 65, 118, 5054, 26, 6, 11, 8, 1104, 8620, 4019, 1936, 8, 2822, 1487, 11724, 5, 1290, 1582, 1208, 1291, 2401, 7, 12, 3055, 57, 3, 9, 2069, 2137, 6, 114, 8, 5034, 94

Para ahorrar algo de tiempo en el laboratorio, submuestreará el conjunto de datos:

In [14]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

Filter:   0%|          | 0/1500 [00:00<?, ? examples/s]

Filter: 100%|██████████| 1500/1500 [00:01<00:00, 1101.08 examples/s]


Compruebe las formas de las tres partes del conjunto de datos:

In [15]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (70, 2)
Validation: (15, 2)
Test: (15, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 70
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15
    })
})


El dataset de salida esta listo para el fine-tunning.

<a name='3'></a>
## 3 - Fine tunning Eficiente de Parámetros (PEFT)

Ahora, realicemos un ajuste fino **PEFT** en lugar del ajuste fino completo, como se hizo anteriormente. PEFT es una forma de ajuste fino de instrucciones mucho más eficiente que el ajuste fino completo, con resultados de evaluación comparables, como verá pronto.

PEFT es un término genérico que incluye **Adaptación de Bajo Rango (LoRA)** y ajuste de prompts (¡que NO ES LO MISMO que ingeniería de prompts!). En la mayoría de los casos, cuando alguien habla de PEFT, se refiere a LoRA. LoRA, a un nivel muy alto, permite al usuario ajustar su modelo utilizando menos recursos computacionales (en algunos casos, una sola GPU). Después del ajuste fino para una tarea, caso de uso o inquilino específico con LoRA, el resultado es que el LLM original permanece sin cambios y surge un nuevo "adaptador LoRA". Este adaptador LoRA es mucho más pequeño que el LLM original: aproximadamente un porcentaje de un solo dígito del tamaño del LLM original (MB vs. GB).

No obstante, en el momento de la inferencia, el adaptador LoRA debe reunirse y combinarse con su LLM original para atender la solicitud de inferencia. La ventaja, sin embargo, es que muchos adaptadores LoRA pueden reutilizar el LLM original, lo que reduce los requisitos generales de memoria al atender múltiples tareas y casos de uso.

<a name='3.1'></a>


### 3.1 - Configuración del modelo PEFT/LoRA para el ajuste fino

Debe configurar el modelo PEFT/LoRA para el ajuste fino con un nuevo adaptador de capa/parámetro. Al usar PEFT/LoRA, se congela el LLM subyacente y solo se entrena el adaptador. Observe la configuración de LoRA a continuación. Observe el hiperparámetro de rango (`r`), que define el rango/dimensión del adaptador que se va a entrenar.

In [16]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

Add LoRA adapter layers/parameters to the original LLM to be trained.

In [17]:
peft_model = get_peft_model(original_model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

Parametros entrenables del modelo: 1,376,256
 Total de parametros del modelo: 78,337,408
 Porcentaje de parametros entrenables 2%


<a name='3.2'></a>
### 3.2 - Train PEFT Adapter

Define training arguments and create `Trainer` instance.

In [18]:
output_dir = f'./peft-finance-instruct-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=1,
    logging_steps=1,
    max_steps=5
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
)

Now everything is ready to train the PEFT adapter and save the model.



In [19]:
peft_trainer.train()
peft_model_path="./peft-dialogue-summary-checkpoint-local"
peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

/home/pablo/Desktop/IA2/mi_env/lib/python3.10/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
1,24.500000
2,16.500000
3,20.125000
4,13.437500
5,15.250000


('./peft-dialogue-summary-checkpoint-local/tokenizer_config.json',
 './peft-dialogue-summary-checkpoint-local/special_tokens_map.json',
 './peft-dialogue-summary-checkpoint-local/spiece.model',
 './peft-dialogue-summary-checkpoint-local/added_tokens.json',
 './peft-dialogue-summary-checkpoint-local/tokenizer.json')

Prepare este modelo añadiendo un adaptador al modelo FLAN-T5 original. Está configurando `is_trainable=False` porque el plan es realizar inferencia únicamente con este modelo PEFT. Si estuviera preparando el modelo para entrenamiento posterior, debería configurar `is_trainable=True`.

In [23]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

peft_model = PeftModel.from_pretrained(peft_model_base,
                                       './peft-dialogue-summary-checkpoint-local/',
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

The number of trainable parameters will be `0` due to `is_trainable=False` setting:

In [25]:
print(print_number_of_trainable_model_parameters(peft_model))

Parametros entrenables del modelo: 0
 Total de parametros del modelo: 78,337,408
 Porcentaje de parametros entrenables 0%


<a name='3.3'></a>
### 3.3 - Evaluate the Model Qualitatively (Human Evaluation)

Make inferences for the same example as in sections [1.3](#1.3) and [2.3](#2.3), with the original model, fully fine-tuned and PEFT model.

In [26]:
dash_line = '-' * 100

# Seleccionar un ejemplo del dataset
index = 200
instruction = dataset['test'][index]['user']
expected_response = dataset['test'][index]['assistant']

# Como no tienes instruct_model, compararemos solo original vs PEFT
# El formato correcto para Finance-Instruct es diferente al de diálogos
prompt = f"""
Financial instruction: {instruction}

Answer: """

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# Generar respuestas
original_model_outputs = original_model.generate(
    input_ids=input_ids, 
    generation_config=GenerationConfig(max_new_tokens=200, num_beams=1)
)
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

peft_model_outputs = peft_model.generate(
    input_ids=input_ids, 
    generation_config=GenerationConfig(max_new_tokens=200, num_beams=1)
)
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

# Mostrar comparación
print(dash_line)
print(f'INSTRUCCIÓN FINANCIERA:\n{instruction}')
print(dash_line)
print(f'RESPUESTA HUMANA (BASELINE):\n{expected_response}')
print(dash_line)
print(f'MODELO ORIGINAL (google/flan-t5-small):\n{original_model_text_output}')
print(dash_line)
print(f'MODELO PEFT (google/flan-t5-small + LoRA):\n{peft_model_text_output}')
print(dash_line)

----------------------------------------------------------------------------------------------------
INSTRUCCIÓN FINANCIERA:
As a math student with a background in applied mathematics and numerics, how can I transition into the field of mathematical finance without prior knowledge in finance or economics?
----------------------------------------------------------------------------------------------------
RESPUESTA HUMANA (BASELINE):
To transition into mathematical finance without prior knowledge in the field, consider the following steps:

* Acquire foundational knowledge: Begin by reading introductory books on finance and economics, such as "Investment Science" by Luenberger.
* Study mathematical finance: Enroll in courses or read books on mathematical finance, such as "Stochastic Calculus for Finance" by Steven Shreve or "Monte Carlo Methods in Financial Engineering" by Glasserman.
* Build a strong statistical foundation: Focus on mastering multivariate statistics and time series ana